In [ ]:
import control
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from dare.utils.nodeconstructorcable import NodeConstructorCable

In [ ]:
def generate_sin(num, t, V_eff=230 * np.sqrt(2), f0 = 50):
    v = list()
    for i in range(num):
        v.append(V_eff * np.sin(2*np.pi * f0 * t+i))
    out = np.vstack(v)
    return out

### Predefined CM

Using NodeConstructor always requires that the number of sources and loads are defined. If you want to specify a certain structure of the connections, you can pass a CM that describes the connections in the node completely. However, it is important that the number of elements in the node matches the structure specified in CM.

In [ ]:
# set up CM

CM = np.array([[0, 0, 1],
               [0, 0, 2],
               [-1, -2, 0]])

Here is an intentionally generated error:

In [ ]:
NodeConstructorCable(3, 2, CM=CM)

3 sources and 2 loads add up to 5 elements in the node, but the CM is only designed for 3 elements.  
  
In the following now a correct definition:

In [ ]:
Grid_S2_L1_2C = NodeConstructorCable(2, 1, CM=CM)

In this case, the parameters of the node are now generated automatically and randomly.

In [ ]:
Grid_S2_L1_2C.parameter

The parameters of the node are summarized in a dict which contains three keys:

In [ ]:
Grid_S2_L1_2C.parameter.keys()

Behind each of these keys there is a list of the respective elements of the node. Let's look at the sources:

In [ ]:
Grid_S2_L1_2C.parameter['source']

In [ ]:
type(Grid_S2_L1_2C.parameter['source'])

This list of the sources contains dicts, which specify the respective source.

In [ ]:
Grid_S2_L1_2C.parameter['source'][0]

### Predefined CM and parameters

Besides the structure, the parameters of the node can be predefined as well. It is important that the previously explained structure is kept. The following code can be used to generate identical values for the elements:

In [ ]:
num_source = 2
num_connections = 2
num_loads = 1

source = dict()
cable = dict()
load = dict()

In [ ]:
#LCL-filter

source['fltr'] = 'LCL'
source['R'] = 10
source['L1'] = 5
source['L2'] = 10
source['C'] = 2

#LC-filter

# source['fltr'] = 'LC'
# source['R'] = 10
# source['L1'] = 5
# source['C'] = 2

In [ ]:
# load

load['R'] = 14

In [ ]:
# cable

Rb = 0.722
Cb = 8*10**-9
Lb = 0.955*10**-3

l = 1

cable['R'] = l * Rb
cable['L'] = l * Lb
cable['C'] = l * Cb

In [ ]:
source_list = list()
cable_list = list()
load_list = list()

for s in range(1, num_source+1):
    source_list.append(source)

for c in range(1, num_connections+1):
    cable_list.append(cable)

for l in range(1, num_loads+1):
    load_list.append(load)

parameter = dict()
parameter['source'] = source_list
parameter['cable'] = cable_list
parameter['load'] = load_list

In [ ]:
Grid_S2_L1_2C = NodeConstructorCable(2, 1, CM=CM, parameter=parameter)

In [ ]:
A, B, C, D = Grid_S2_L1_2C.get_sys()

In [ ]:
sys = control.ss(A, B, C, D)

In [ ]:
# define time vector
ts = 1e-4
t_end = 0.005
steps = int(1/ts)
t = np.arange(0, t_end+ts, ts)
num_samples = len(t)

# generate init state
x0 = np.zeros((A.shape[0],1))

# simple input signal of constant 230V from all sources
u = np.array([230]).repeat(Grid_S2_L1_2C.num_source)[:,None] * np.ones((Grid_S2_L1_2C.num_source,len(t)))

T, yout, xout = control.forced_response(sys, T=t, U=u, X0=x0, return_x=True, squeeze=True)

plt.plot(t, xout[1], label='$v_1$')
plt.xlabel(r'$t\,/\,\mathrm{s}$')
plt.ylabel('$v_{\mathrm{1}}\,/\,\mathrm{V}$')
plt.title('Plot current $v_1$')
plt.legend()
plt.grid()
plt.show()